In [ ]:
import pandas as pd
import numpy as np

# Membuat data dummy
np.random.seed(0)

# Nama daerah
daerah = ['Daerah_A', 'Daerah_B', 'Daerah_C', 'Daerah_D', 'Daerah_E']
n_samples = 10000

# Fitur geografis
curah_hujan = np.random.uniform(50, 300, n_samples)  # Curah hujan dalam mm
suhu = np.random.uniform(15, 35, n_samples)         # Suhu dalam derajat Celsius
ketinggian = np.random.uniform(0, 2000, n_samples)  # Ketinggian dalam meter

# Status potensi banjir (0: Aman, 1: Waspada, 2: Siaga, 3: Awas)
status_banjir = np.random.randint(0, 4, n_samples)

# Membuat DataFrame
data = pd.DataFrame({
    'Daerah': np.random.choice(daerah, n_samples),
    'Curah Hujan': curah_hujan,
    'Suhu': suhu,
    'Ketinggian': ketinggian,
    'Status Banjir': status_banjir
})

data.head()


In [ ]:
# Membuat data uji tanpa status potensi banjir
n_test_samples = 100
test_data = pd.DataFrame({
    'Daerah': np.random.choice(daerah, n_test_samples),
    'Curah Hujan': np.random.uniform(50, 300, n_test_samples),
    'Suhu': np.random.uniform(15, 35, n_test_samples),
    'Ketinggian': np.random.uniform(0, 2000, n_test_samples)
})

test_data.head()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Persiapan data untuk model
X = data[['Curah Hujan', 'Suhu', 'Ketinggian']].values
y = data['Status Banjir'].values

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Membagi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

# Implementasi Levenberg-Marquardt untuk model logistik
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def loss_function(params, X, y):
    weights = params[:-1].reshape((X.shape[1], 1))
    bias = params[-1]
    predictions = sigmoid(X.dot(weights) + bias)
    loss = -np.mean(y * np.log(predictions + 1e-8) + (1 - y) * np.log(1 - predictions + 1e-8))
    return loss

def gradient(params, X, y):
    weights = params[:-1].reshape((X.shape[1], 1))
    bias = params[-1]
    predictions = sigmoid(X.dot(weights) + bias)
    error = predictions - y.reshape(-1, 1)
    grad_weights = X.T.dot(error) / X.shape[0]
    grad_bias = np.sum(error) / X.shape[0]
    return np.concatenate([grad_weights.ravel(), [grad_bias]])

def levenberg_marquardt(X, y, initial_params, num_iterations=100):
    params = initial_params
    mu = 0.01
    lambda_factor = 10
    
    for i in range(num_iterations):
        grad = gradient(params, X, y)
        loss = loss_function(params, X, y)
        
        # Update step
        params -= mu * grad
        
        # Check if loss decreases
        new_loss = loss_function(params, X, y)
        if new_loss < loss:
            mu /= lambda_factor
        else:
            mu *= lambda_factor
    
    return params

# Inisialisasi parameter
initial_params = np.random.randn(X_train.shape[1] + 1)

# Menjalankan LM
optimized_params = levenberg_marquardt(X_train, y_train, initial_params)
print("Optimized Parameters:", optimized_params)


In [ ]:
# Mapping kategori
status_mapping = {0: 'Aman', 1: 'Waspada', 2: 'Siaga', 3: 'Awas'}

# Fungsi untuk memetakan hasil prediksi ke kategori
def map_predictions(predictions):
    return [status_mapping[int(pred)] for pred in predictions]

# Prediksi pada data uji
predictions = predict(X_test_scaled, optimized_params)
predicted_statuses = map_predictions(predictions)

# Menampilkan hasil
test_data['Predicted Status'] = predicted_statuses
print(test_data)
